In [1]:
import os
import csv
import pandas as pd
import pandas.io.sql as psql
import datetime as dt
import numpy as np
import warnings


import gspread
import gspread_dataframe as gd
import psycopg2 as pg

In [2]:
# input is from a Gsheet maintained by MKT team
gc1 = gspread.service_account('C:/service_account.json')
sh = gc1.open_by_url('https://docs.google.com/spreadsheets/d/1pdHL9O0rmrjzpJm8WFXdIecjOuuYBVbb-aFpK5yNqAE/edit#gid=0')

sh1 = sh.worksheet("Campaign Dates")

from datetime import date

start_date = date.fromisoformat('2023-09-01')
start_date = dt.datetime.strftime(start_date, '%Y-%m-%d')

In [3]:
# https://lookerstudio.google.com/u/0/reporting/b6b60311-0590-4ca4-9c7c-494d9420f1b3/page/lxjqC 
tw = pd.read_csv('C:\\Directory_Here\\TW Traffic.csv')
tw['Country'] = 'TW'
sg = pd.read_csv('C:\\Directory_Here\\SG Traffic.csv')
sg['Country'] = 'SG'
my = pd.read_csv('C:\\Directory_Here\\MY Traffic.csv')
my['Country'] = 'MY'
au = pd.read_csv('C:\\Directory_Here\\AU Traffic.csv')
au['Country'] = 'AU'
nz = pd.read_csv('C:\\Directory_Here\\NZ Traffic.csv')
nz['Country'] = 'NZ'
us = pd.read_csv('C:\\Directory_Here\\US Traffic.csv')
us['Country'] = 'US'
hk = pd.read_csv('C:\\Directory_Here\\HK Traffic.csv')
hk['Country'] = 'HK'
ph = pd.read_csv('C:\\Directory_Here\\PH Traffic.csv')
ph['Country'] = 'PH'
indo = pd.read_csv('C:\\Directory_Here\\ID Traffic.csv')
indo['Country'] = 'ID'
# cad = pd.read_csv('C:\\Directory_Here\\CA Traffic.csv')
# cad['Country'] = 'CA'
uk = pd.read_csv('C:\\Directory_Here\\UK Traffic.csv')
uk['Country'] = 'UK'

In [4]:
DbMain = pg.connect(
     host="HOST",
     database="DB",
     user="USER",
     password="PASSWORD")

# If Campaign Specific - Redemptions

In [6]:
mothersday = psql.read_sql('''
with code_usage as (SELECT to_char(b.created_at + interval '8 hours', 'DD/MM/YYYY') as date, c.name,
count(DISTINCT b.id) as transactions FROM transaction_buyer b 
LEFT JOIN country c ON b.buyer_country_id = c.id
LEFT JOIN promocode p ON b.promocode_id = p.id
left join transaction_seller s
on b.transaction_seller_id = s.id
where b."type" = 'transaction'
and b.status not in ('buy_failed', 'canceled','failed','payment_failed', 'sell_failed')
and ((COALESCE(b.nature,'default') in ('Pre Order')) and procure_status not in ('check_procurement_status','null',
'to_procure','bidding','to_complete_buy', 'contact_platform')
or COALESCE(b.nature,'default') not in  ('Pre Order'))
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853) 
and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null) and p.code in ('JUNE6', 'JUNE12')
and date(b.created_at + interval '8 hours' ) >= date('%s')
GROUP BY 1,2
ORDER BY 1,2),

txn_table as (SELECT to_char(b.created_at + interval '8 hours', 'DD/MM/YYYY') as date, c.name,
count(DISTINCT b.id) as transactions FROM transaction_buyer b 
LEFT JOIN country c ON b.buyer_country_id = c.id
LEFT JOIN promocode p ON b.promocode_id = p.id
left join transaction_seller s
on b.transaction_seller_id = s.id
where b."type" = 'transaction'
and b.status not in ('buy_failed', 'canceled','failed','payment_failed', 'sell_failed')
and ((COALESCE(b.nature,'default') in ('Pre Order')) and procure_status not in ('check_procurement_status','null',
'to_procure','bidding','to_complete_buy', 'contact_platform')
or COALESCE(b.nature,'default') not in  ('Pre Order'))
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853) 
and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null) 
and date(b.created_at + interval '8 hours' ) >= date('%s')
GROUP BY 1,2
ORDER BY 1,2)

SELECT code_usage.date, code_usage.name, code_usage.transactions as code_usage_transactions, 
txn_table.transactions as txn_table_transactions 
from code_usage 
LEFT JOIN txn_table ON code_usage.date = txn_table.date AND code_usage.name = txn_table.name
''' % (start_date, start_date), DbMain)
mothersday['redemption'] = mothersday['code_usage_transactions']/mothersday['txn_table_transactions']*100
mothersday

C:\Users\novel\AppData\Local\Temp\ipykernel_18460\3227641271.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mothersday = psql.read_sql('''


,date,name,code_usage_transactions,txn_table_transactions,redemption


# Generic Promocode Usage

In [7]:
camp_info = gd.get_as_dataframe(sh1)[['Campaign Name', 'Campaign Code', 'Start Date', 'End Date']]
camp_info = camp_info.dropna()

In [8]:
camp_codes = gd.get_as_dataframe(sh1)['Campaign Code']
camp_codes = camp_codes.dropna()
camp_codes = tuple(camp_codes)

def combine_str(items):
    result = "', '".join(str(item) for item in items[:-1]) #exclude last one if not last item will have ", " after it as well
    return "'" + result + "', '" + str(items[-1]) + "'" #add back the last one

camp_codes = "(" + combine_str(camp_codes) + ")"
camp_codes

"('NS9APP', 'NS10DAY', 'NS10APP', 'OFFERS', 'TNF15', 'NS11DAY', 'NSNOV', 'HH11', 'FREE11', 'AF5NOV', 'ECO5', 'BFFREE', 'CM5OFF', 'VIC20', 'BF5OFF', 'BF7OFF', 'CMFREE', 'CM5OFF', 'BFFREE', 'NS12', 'NSAB12', 'NSAB13', 'HH12', 'NS12DAY', 'XMHH26', 'XMHH25', 'XMHH24', 'XMNB6', 'XMNB5', 'NSJAN', 'NS1DAY', 'NYRNB', 'NYRAB', 'NYR10', 'NYR24', 'FIRSTLPAPP', 'FIRSTLP150', 'UGG5', 'PF5OFF', 'BS3OFF', 'NS5OFF', 'NS8OFF', 'GOTCHA05', 'MAR5OFF', 'MAR20OFF', 'MAR8OFF', 'IWD8', 'IWD12OFF', 'APRILAGAIN', 'APR8OFF', 'EASTER', 'RAYAREADY', 'RAYAGS', 'RAYA15', 'EARTH5', 'FREESHIPPING', 'MAYDAY', 'MAYDAY10', 'MOTHERSDAY', 'MOM', 'WELCOMENEW', 'FREESHIPPING')"

In [10]:
campaign_transactions = psql.read_sql('''
SELECT p.code as campaign_code, c.name, 
count(DISTINCT b.id) as transactions FROM transaction_buyer b 
LEFT JOIN currency c ON b.buyer_currency_id = c.id
LEFT JOIN promocode p ON b.promocode_id = p.id
left join transaction_seller s
on b.transaction_seller_id = s.id
where b."type" = 'transaction'
and b.status not in ('buy_failed', 'canceled','failed','payment_failed', 'sell_failed')
and ((COALESCE(b.nature,'default') in ('Pre Order')) and procure_status not in ('check_procurement_status','null',
'to_procure','bidding','to_complete_buy', 'contact_platform')
or COALESCE(b.nature,'default') not in  ('Pre Order'))
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853) 
and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null) and p.code in %s
and date(b.created_at + interval '8 hours' ) >= date('%s')
GROUP BY 1,2
ORDER BY 1,2
''' % (camp_codes, start_date), DbMain)
campaign_transactions

C:\Users\novel\AppData\Local\Temp\ipykernel_38224\2952906953.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  campaign_transactions = psql.read_sql('''


,campaign_code,name,transactions
0,AF5NOV,AUD,13
1,AF5NOV,NZD,19
2,APR8OFF,MYR,2
3,APR8OFF,SGD,13
4,APRILAGAIN,AUD,3
...,...,...,...
152,XMNB6,AUD,3
153,XMNB6,MYR,1
154,XMNB6,NZD,4
155,XMNB6,SGD,9


In [11]:
campaign_transactions = pd.merge(camp_info, campaign_transactions, how = 'left', left_on = 'Campaign Code', right_on = 'campaign_code')
campaign_transactions.drop(columns = ['campaign_code'], inplace=True)
unique_names = campaign_transactions.groupby('Campaign Code')['name'].apply(lambda x: list(set(x))).reset_index()

campaign_transactions = campaign_transactions.merge(unique_names, on='Campaign Code', how='left')
campaign_transactions.rename(columns={'name_x': 'name', 'name_y': 'unique_names'}, inplace=True)

sh2 = sh.worksheet('Campaign Transactions')
gd.set_with_dataframe(sh2, campaign_transactions)

In [12]:
camp_info['Start Date'] = pd.to_datetime(camp_info['Start Date'])
camp_info['End Date'] = pd.to_datetime(camp_info['End Date'])

unique_dates = []
day_buffer = 4

for index, row in camp_info.iterrows():
    date_range = pd.date_range(start=row['Start Date'] - pd.Timedelta(days = day_buffer), end=row['End Date'] + pd.Timedelta(days = day_buffer))
    formatted_dates = [date.strftime('%m/%d/%Y') for date in date_range]
    unique_dates.extend(date for date in formatted_dates if date not in unique_dates)

unique_dates.sort()
unique_dates_string = str(unique_dates).replace('[', '(').replace(']', ')')

['01/01/2024', '01/02/2024', '01/03/2024', '01/04/2024', '01/05/2024', '01/06/2024', '01/07/2024', '01/08/2024', '01/09/2024', '01/10/2024', '01/11/2024', '01/12/2024', '01/13/2024', '01/14/2024', '01/15/2024', '01/16/2024', '01/17/2024', '01/18/2024', '01/19/2024', '01/20/2024', '01/21/2024', '01/22/2024', '01/23/2024', '01/24/2024', '01/25/2024', '01/26/2024', '01/27/2024', '01/28/2024', '01/29/2024', '01/30/2024', '01/31/2024', '02/01/2024', '02/02/2024', '02/03/2024', '02/04/2024', '02/05/2024', '02/06/2024', '02/16/2024', '02/17/2024', '02/18/2024', '02/19/2024', '02/20/2024', '02/21/2024', '02/22/2024', '02/23/2024', '02/24/2024', '02/25/2024', '02/26/2024', '02/27/2024', '02/28/2024', '02/29/2024', '03/01/2024', '03/02/2024', '03/03/2024', '03/04/2024', '03/05/2024', '03/06/2024', '03/07/2024', '03/08/2024', '03/09/2024', '03/10/2024', '03/11/2024', '03/12/2024', '03/13/2024', '03/14/2024', '03/15/2024', '03/16/2024', '03/17/2024', '03/18/2024', '03/19/2024', '03/30/2024', '03/3

# Daily Transactions

In [13]:
daily_transactions = psql.read_sql(
'''select to_char(b.created_at + interval '8 hours','MM/DD/YYYY') as order_date,
c.name,
count (b.id) as transactions FROM transaction_buyer b 
LEFT JOIN currency c ON b.buyer_currency_id = c.id
LEFT JOIN promocode p ON b.promocode_id = p.id
left join transaction_seller s
on b.transaction_seller_id = s.id
where b."type" = 'transaction'
and b.status not in ('buy_failed', 'canceled','failed','payment_failed', 'sell_failed')
and ((COALESCE(b.nature,'default') in ('Pre Order')) and procure_status not in ('check_procurement_status','null',
'to_procure','bidding','to_complete_buy', 'contact_platform')
or COALESCE(b.nature,'default') not in  ('Pre Order'))
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853) 
and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null) and to_char(b.created_at + interval '8 hours','MM/DD/YYYY') in %s
GROUP BY 1,2
ORDER BY 1,2
''' % (unique_dates_string), DbMain)

C:\Users\novel\AppData\Local\Temp\ipykernel_38224\64114670.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  daily_transactions = psql.read_sql(


In [14]:
sh3 = sh.worksheet('Daily Transactions')
gd.set_with_dataframe(sh3, daily_transactions)

# PC3

In [15]:
average_pc3 = psql.read_sql(
'''
with 
gross_gmv as (
select to_char(b.created_at + interval '8 hours','MM/DD/YYYY') as date,
b.ref as trxns_buyer_ref,
b.id as gross_id,
b.buyer_id as gross_buyer_id,
u.email as seller_email,
c.name as buyer_country,
cr.name as seller_country,
cs.name as processing_country,
crn.name as buyer_currency,
sum(COALESCE(list_price_local/seller_currency_rate,0)
+ COALESCE(fee_delivery/buyer_currency_rate,0)
+ COALESCE(fee_delivery_discount/buyer_currency_rate,0)
+ COALESCE(fee_delivery_insurance/buyer_currency_rate,0)
+ COALESCE(fee_delivery_instant/buyer_currency_rate,0)
+ case when COALESCE(offer_price_local/buyer_currency_rate,0) > COALESCE(list_price_local/seller_currency_rate,0)
    then COALESCE(offer_price_local/buyer_currency_rate,0) - COALESCE(list_price_local/seller_currency_rate,0)
    else 0 end
+ COALESCE(fee_add_on/buyer_currency_rate,0)
+ COALESCE(fee_processing_buy/buyer_currency_rate,0) 
- COALESCE(total_tax_reportable/buyer_currency_rate,0)) as gross_gmv
from transaction_buyer b
left join transaction_seller s
on b.transaction_seller_id = s.id
left join country c
on c.id = b.buyer_country_id
left join country cr
on cr.id = s.seller_country_id
left join country cs
on cs.id = s.processing_country_id
left join currency crn
on crn.id = b.buyer_currency_id
left join "user" u
on s.seller_id = u.id
where to_char(b.created_at + interval '8 hours','MM/DD/YYYY') in %s
and b."type" = 'transaction'
and b.status not in ('payment_failed')
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853) 
and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null)
group by 1,2,3,4,5,6,7,8,9
order by 1
),

gmv as(
select to_char(b.created_at + interval '8 hours','MM/DD/YYYY') as date,
b.id as net_id,
b.buyer_id as net_buyer_id,
count (distinct case when buyer_country_id = processing_country_id then b.id else null end) as domestic_transactions,
sum(COALESCE(list_price_local/seller_currency_rate,0)
+ COALESCE(fee_delivery/buyer_currency_rate,0)
+ COALESCE(fee_delivery_discount/buyer_currency_rate,0)
+ COALESCE(fee_delivery_insurance/buyer_currency_rate,0)
+ COALESCE(fee_delivery_instant/buyer_currency_rate,0)
+ case when COALESCE(offer_price_local/buyer_currency_rate,0) > COALESCE(list_price_local/seller_currency_rate,0)
    then COALESCE(offer_price_local/buyer_currency_rate,0) - COALESCE(list_price_local/seller_currency_rate,0)
    else 0 end
+ COALESCE(fee_add_on/buyer_currency_rate,0)
+ COALESCE(fee_processing_buy/buyer_currency_rate,0) 
- COALESCE(total_tax_reportable/buyer_currency_rate,0)) as gmv,
sum(b.promocode_value/buyer_currency_rate) as promo_value
from transaction_buyer b
left join transaction_seller s
on b.transaction_seller_id = s.id
where to_char(b.created_at + interval '8 hours','MM/DD/YYYY') in %s
and b."type" = 'transaction'
and b.status not in ('buy_failed', 'canceled','failed','payment_failed', 'sell_failed')
and ((COALESCE(b.nature,'default') in ('Pre Order')) and procure_status not in ('check_procurement_status','null','to_procure','bidding','to_complete_buy', 'contact_platform')
or COALESCE(b.nature,'default') not in  ('Pre Order'))
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853) 
and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null)
group by 1,2,3
order by 1),

payment_g as (
select to_char(b.created_at + interval '8 hours','MM/DD/YYYY') as date,
b.id,
sum(COALESCE(
case when payment_method in ('stripe','stripe_alipay','stripe_grabpay','rapyd_paynow','hoolah','adyen_card')
then payment_fee+COALESCE(payment_fee_additional,0)  else (payment_fee+COALESCE(payment_fee_additional,0))/buyer_currency_rate end ,0)
+COALESCE(fee_delivery_payment_fee,0)
+COALESCE(fee_delivery_insurance_payment_fee,0)
+COALESCE(payout_balance_debit_discount_value/buyer_currency_rate,0)) as payment_gateway
from transaction_buyer b
left join transaction_seller s
on b.transaction_seller_id = s.id
where to_char(b.created_at + interval '8 hours','MM/DD/YYYY') in %s
and b."type" = 'transaction'
and b.status not in ('buy_failed', 'canceled','failed','payment_failed', 'sell_failed')
and ((COALESCE(b.nature,'default') in ('Pre Order')) and procure_status not in ('check_procurement_status','null','to_procure','bidding','to_complete_buy', 'contact_platform')
or COALESCE(b.nature,'default') not in  ('Pre Order'))
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853) and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null)
and payment_fee <> 'nan'
group by 1,2
order by 1
),

seller_campaign as (
select to_char(b.created_at + interval '8 hours','MM/DD/YYYY') as date,
b.id,
sum(COALESCE(fee_selling_discount/seller_currency_rate,0)
+ COALESCE(fee_shipping_discount/seller_currency_rate,0)) as seller_campaign
from transaction_buyer b
left join transaction_seller s
on b.transaction_seller_id = s.id
where to_char(b.created_at + interval '8 hours','MM/DD/YYYY') in %s
and b."type" = 'transaction'
and b.status not in ('buy_failed', 'canceled','failed','payment_failed', 'sell_failed')
and ((COALESCE(b.nature,'default') in ('Pre Order')) and procure_status not in ('check_procurement_status','null','to_procure','bidding','to_complete_buy', 'contact_platform')
or COALESCE(b.nature,'default') not in  ('Pre Order'))
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853)
and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null)
group by 1,2
order by 1
),

other_subsidy as (
select to_char(b.created_at + interval '8 hours','MM/DD/YYYY') as date,
b.id,
sum(COALESCE(b.capture_subsidy,0)
+ COALESCE(s.capture_subsidy,0)) as other_subsidy
from transaction_buyer b
left join transaction_seller s
on b.transaction_seller_id = s.id
where to_char(b.created_at + interval '8 hours','MM/DD/YYYY') in %s
and b."type" = 'transaction'
and b.status not in ('buy_failed', 'canceled','failed','payment_failed', 'sell_failed')
and ((COALESCE(b.nature,'default') in ('Pre Order')) and procure_status not in ('check_procurement_status','null','to_procure','bidding','to_complete_buy', 'contact_platform')
or COALESCE(b.nature,'default') not in  ('Pre Order'))
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853) 
and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null)
group by 1,2
order by 1
),

logistics as (
select to_char(b.created_at + interval '8 hours','MM/DD/YYYY') as date,
b.id,
sum(COALESCE(b.delivery_cost_estimate,0)) as buyer_logistics,
sum(COALESCE(s.shipping_cost_estimate,0)) as seller_logistics
from transaction_buyer b
left join transaction_seller s
on b.transaction_seller_id = s.id
where to_char(b.created_at + interval '8 hours','MM/DD/YYYY') in %s
and b."type" = 'transaction'
and b.status not in ('buy_failed', 'canceled','failed','payment_failed', 'sell_failed')
and ((b.nature in ('Pre Order')) and
    procure_status not in ('null','to_procure','bidding','to_complete_buy')
    or b.nature not in  ('Pre Order'))
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853) 
and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null)
group by 1,2
order by 1
),

pre_order as (select to_char(b.created_at + interval '8 hours','MM/DD/YYYY') as date,
b.id,
sum(COALESCE(s.procure_cost/procure_cost_currency_rate,0) +
    COALESCE(s.procure_forwarder_cost,0) +
case when COALESCE(b.fee_add_on/buyer_currency_rate,0) > 0 then 5 else 0 end )
as pre_order_cogs
from transaction_buyer b
left join transaction_seller s
on b.transaction_seller_id = s.id
where to_char(b.created_at + interval '8 hours','MM/DD/YYYY') in %s
and b."type" = 'transaction'
and b.status not in ('buy_failed', 'canceled', 'failed','payment_failed', 'sell_failed')
and ((b.nature in ('Pre Order')) and procure_status not in ('check_procurement_status','null','to_procure','bidding','to_complete_buy', 'contact_platform') or b.nature not in  ('Pre Order'))
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853) and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null)
and b.nature = 'Pre Order'
group by 1,2
order by 1),

non_pre_order as (select to_char(b.created_at + interval '8 hours','MM/DD/YYYY') as date,
b.id,
sum(COALESCE(inventory_procure_cost,payout_amount,0) +
case when COALESCE(b.fee_add_on/buyer_currency_rate,0) > 0 then 5 else 0 end )
as cogs
from transaction_buyer b
left join transaction_seller s
on b.transaction_seller_id = s.id
where to_char(b.created_at + interval '8 hours','MM/DD/YYYY') in %s
and b."type" = 'transaction'
and b.status not in ('buy_failed', 'canceled', 'failed','payment_failed', 'sell_failed')
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853) and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null)
and b.nature <> 'Pre Order'
group by 1,2
order by 1),

promo_gmv as (select to_char(b.created_at + interval '8 hours','MM/DD/YYYY') as date,
p.type,
b.id,
sum(COALESCE(b.promocode_value/b.buyer_currency_rate,0)) as promocode_value
from transaction_buyer b
left join transaction_seller s
on b.transaction_seller_id = s.id
left join promocode p
on b.promocode_id = p.id
where to_char(b.created_at + interval '8 hours','MM/DD/YYYY') in %s
and b."type" = 'transaction'
and b.status not in ('buy_failed', 'canceled', 'failed','payment_failed', 'sell_failed')
and ((COALESCE(b.nature,'default') in ('Pre Order')) and procure_status not in ('check_procurement_status','null','to_procure','bidding','to_complete_buy', 'contact_platform')
or COALESCE(b.nature,'default') not in  ('Pre Order'))
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853) 
and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null)
group by 1,2,3),

give_away as ( select to_char(b.created_at + interval '8 hours','MM/DD/YYYY') as date,
b.id,
sum(COALESCE(b.promocode_value/b.buyer_currency_rate,0)) as giveaway_promo
from transaction_buyer b
left join transaction_seller s
on b.transaction_seller_id = s.id
left join promocode p
on b.promocode_id = p.id
where to_char(b.created_at + interval '8 hours','MM/DD/YYYY') in %s
and b."type" = 'transaction'
and b.status not in ('buy_failed', 'canceled', 'failed','payment_failed', 'sell_failed')
and ((COALESCE(b.nature,'default') in ('Pre Order')) and procure_status not in ('check_procurement_status','null','to_procure','bidding','to_complete_buy', 'contact_platform')
or COALESCE(b.nature,'default') not in  ('Pre Order'))
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853) 
and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null)
and p.type = 'giveaway'
group by 1,2),

add_on_discounts as (select to_char(b.created_at + interval '8 hours','MM/DD/YYYY') as date,
b.id,
sum(add_on_quantity*5) as addon_discount
from transaction_buyer b
left join transaction_buyer_add_on tbao
on b.id = tbao.transaction_buyer_id
left join transaction_seller s
on b.transaction_seller_id = s.id
where b.fee_add_on=0
and to_char(b.created_at + interval '8 hours','MM/DD/YYYY') in %s
and b."type" = 'transaction'
and b.status not in ('buy_failed', 'canceled', 'failed',
    'payment_failed', 'sell_failed')
and ((COALESCE(b.nature,'default') in ('Pre Order')) and procure_status not in ('check_procurement_status','null','to_procure','bidding','to_complete_buy', 'contact_platform')
or COALESCE(b.nature,'default') not in  ('Pre Order'))
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853) 
and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null)
group by 1,2),

to_storage_trxns as (
select to_char(b.created_at + interval '8 hours','MM/DD/YYYY') as date,
b.id as to_storage_id,
count (b.id) as to_storage_transactions

from transaction_buyer b
left join transaction_seller s
on b.transaction_seller_id = s.id
where to_char(b.created_at + interval '8 hours','MM/DD/YYYY') in %s
and b."type" = 'transaction'
and b.status not in ('buy_failed', 'canceled','failed','payment_failed', 'sell_failed')
and ((COALESCE(b.nature,'default') in ('Pre Order')) and procure_status not in ('check_procurement_status','null','to_procure','bidding','to_complete_buy', 'contact_platform')
or COALESCE(b.nature,'default') not in  ('Pre Order'))
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853) 
and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null)
and b.deliver_to = 'storage' 
group by 1,2
order by 1
),

processing_cost as (
select to_char(b.created_at + interval '8 hours','MM/DD/YYYY') as date,
b.id,
SUM(
        CASE 
            WHEN c1.shortcode = 'SG' THEN 5.5158
            WHEN c1.shortcode = 'TW' THEN 6.2549
            WHEN c1.shortcode = 'JP' THEN 5.62389
            WHEN c1.shortcode = 'ID' THEN 4.8125
            WHEN c1.shortcode = 'AU' THEN 5.813
            WHEN c1.shortcode = 'MY' THEN 5.6689
            WHEN c1.shortcode = 'HK' THEN 6.80
            ELSE 0 
        END
    ) AS processing_cost
from transaction_buyer b
left join transaction_seller s
on b.transaction_seller_id = s.id
LEFT JOIN country c1 
ON processing_country_id = c1.id
where to_char(b.created_at + interval '8 hours','MM/DD/YYYY') in %s
and b."type" = 'transaction'
and b.status not in ('buy_failed', 'canceled','failed','payment_failed', 'sell_failed')
and ((COALESCE(b.nature,'default') in ('Pre Order')) and procure_status not in ('check_procurement_status','null','to_procure','bidding','to_complete_buy', 'contact_platform')
or COALESCE(b.nature,'default') not in  ('Pre Order'))
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853) 
and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null)
group by 1,2
order by 1
)

select 
g.*,
gr.net_id, 
gr.net_buyer_id, 
coalesce(gr.domestic_transactions, 0) as domestic_transactions, 
coalesce(gr.gmv, 0) as gmv, 
coalesce(gr.promo_value, 0) as promo_value,
coalesce(pg.payment_gateway, 0) as payment_gateway,
coalesce(sc.seller_campaign, 0) as seller_campaign,
coalesce(os.other_subsidy, 0) as other_subsidy, 
coalesce(lo.buyer_logistics, 0) as buyer_logistics, 
coalesce(lo.seller_logistics, 0) as seller_logistics, 
coalesce(po.pre_order_cogs, 0) as pre_order_cogs,
coalesce(npo.cogs, 0) as cogs,
coalesce(ad.addon_discount, 0) as addon_discount,
coalesce(ga.giveaway_promo, 0) as giveaway_promo,
coalesce(to_storage_transactions, 0) as to_storage_transactions,
coalesce((sum(case when pr.type in ('customer_service','refund') then promocode_value else 0 end)), 0) as customer_service_promo,
coalesce((sum(case when pr.type = 'novelship_employee' then promocode_value else 0 end)), 0) as hr_promo,
coalesce(sum(processing_cost), 0) as processing_cost
from gross_gmv g
left join gmv gr on gr.date = g.date and gr.net_id = g.gross_id
left join payment_g pg on pg.date = g.date and pg.id = g.gross_id
left join seller_campaign sc on sc.date = g.date and sc.id = g.gross_id
left join other_subsidy os on os.date = g.date and os.id = g.gross_id
left join logistics lo on lo.date = g.date and lo.id = g.gross_id
left join pre_order po on po.date = g.date and po.id = g.gross_id
left join non_pre_order npo on npo.date = g.date and npo.id = g.gross_id
left join promo_gmv pr on pr.date = g.date and pr.id = g.gross_id
left join add_on_discounts ad on ad.date = g.date and ad.id = g.gross_id
left join give_away as ga on ga.date = g.date and ga.id = g.gross_id
left join to_storage_trxns as tst on tst.date = g.date and tst.to_storage_id = g.gross_id 
left join processing_cost as pct on pct.date = g.date and pct.id = g.gross_id
GROUP by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
''' % (unique_dates_string, unique_dates_string, unique_dates_string, 
       unique_dates_string,unique_dates_string, unique_dates_string, 
       unique_dates_string, unique_dates_string,unique_dates_string, 
       unique_dates_string, unique_dates_string, unique_dates_string,
       unique_dates_string), DbMain)

C:\Users\novel\AppData\Local\Temp\ipykernel_38224\630727854.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  average_pc3 = psql.read_sql(


In [16]:
df = average_pc3
object_columns = df.select_dtypes(include="object").columns.tolist()
num_columns = df.select_dtypes(exclude="object").columns.tolist()
df[object_columns] = df[object_columns].fillna("Others")
df[num_columns] = df[num_columns].fillna(0)

df = df.rename({"gross_id" : "trxns_buyer_id", "gross_buyer_id" : "buyer_id"}, axis=1)
df["total_cogs"] = df["pre_order_cogs"] + df["cogs"]
df["pc1"] = df["gmv"] - df["total_cogs"]
df["pc1_perc"] = df["pc1"] / df["gmv"]

df["total_logistic"] = df["buyer_logistics"] + df["seller_logistics"]
df["fullfilment_cost"] = df["total_logistic"] + df["payment_gateway"] \
    + df["customer_service_promo"] + df["processing_cost"]
df["pc2"] = df["pc1"] - df["fullfilment_cost"]
df["pc2_perc"] = df["pc2"] / df["gmv"]

df['promo_value'] = df['promo_value'] - \
                        df['customer_service_promo'] - df['hr_promo']\
                        + df['addon_discount']
df["marketing_cost"] = df["promo_value"] + df["seller_campaign"] + df["other_subsidy"]
df["pc3"] = df["pc2"] - df["marketing_cost"]
df["pc3_perc"] = df["pc3"] / df["gmv"]
df = df[["date", "trxns_buyer_ref", "trxns_buyer_id",
                                    "buyer_id", "gmv", "total_cogs", "pc1", "pc1_perc",
                                    "total_logistic", "payment_gateway", "customer_service_promo", "processing_cost","pc2", "pc2_perc",
                                    "promo_value", "seller_campaign", "other_subsidy", "pc3", "pc3_perc",
                                    "buyer_country", "seller_country", "processing_country", "buyer_currency"]]
df = df.groupby(['date']).agg(pc3=('pc3', 'sum'),
                                    trxns=('trxns_buyer_id', 'count'),
                                    gmv=('gmv', 'sum'),
                                    cogs=('total_cogs', 'sum'),
                                    pc1=('pc1', 'sum')).reset_index()
df['pc3_perc'] = df['pc3']/df['gmv']

sh4 = sh.worksheet('PC Transactions')
gd.set_with_dataframe(sh4, df)

# New Buyers

In [17]:
new_buyer = psql.read_sql('''with first_buy as (select
buyer_id,min(b.created_at + interval '8 hours') as first_order_date,
min(b.id) as first_order_id
from transaction_buyer b
left join transaction_seller s
on b.transaction_seller_id = s.id
where
b."type" = 'transaction' and
 b.status not in ('buy_failed', 'canceled', 'failed','payment_failed', 'sell_failed')
and b.payment_method!='credit'
and buyer_id not in (12409, 20095, 1804, 19022, 87853) and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
and (seller_id not in (12409, 12698) or seller_id is null)
and ((COALESCE(b.nature,'default') in ('Pre Order')) and procure_status not in ('check_procurement_status','null',
'to_procure','bidding','to_complete_buy', 'contact_platform')
or COALESCE(b.nature,'default')  not in  ('Pre Order'))
group by 1
order by 1)

select to_char(b.created_at + interval '8 hours','MM/DD/YYYY') as order_date,
count(distinct f.buyer_id) as new_buyers,
sum(b.promocode_value/buyer_currency_rate) as new_buyer_promo_value,
sum(case when b.promocode_value > 0 then COALESCE(list_price_local/seller_currency_rate,0)
+ COALESCE(fee_delivery/buyer_currency_rate,0)
+ COALESCE(fee_delivery_discount/buyer_currency_rate,0)
+ COALESCE(fee_delivery_insurance/buyer_currency_rate,0)
+ COALESCE(fee_delivery_instant/buyer_currency_rate,0)
+ case when COALESCE(offer_price_local/buyer_currency_rate,0) > COALESCE(list_price_local/seller_currency_rate,0)
	then COALESCE(offer_price_local/buyer_currency_rate,0) - COALESCE(list_price_local/seller_currency_rate,0)
	else 0 end
+ COALESCE(fee_add_on/buyer_currency_rate,0)
+ COALESCE(fee_processing_buy/buyer_currency_rate,0)
 - COALESCE(total_tax_reportable/buyer_currency_rate,0) else 0 end) as new_buyer_promo_gmv,
sum(COALESCE(b.capture_subsidy,0)) as new_buyer_autocaptue,
sum(COALESCE(s.capture_subsidy,0)) as new_buyer_seller_autocaptue
from first_buy f
left join transaction_buyer b
on b.id = f.first_order_id
left join transaction_seller s
on b.transaction_seller_id = s.id
where to_char(b.created_at + interval '8 hours','MM/DD/YYYY') in %s
group by 1
order by 1 ''' % (unique_dates_string), DbMain)

C:\Users\novel\AppData\Local\Temp\ipykernel_38224\3610211299.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  new_buyer = psql.read_sql('''with first_buy as (select


In [18]:
sh6 = sh.worksheet('New Buyer Transactions')
gd.set_with_dataframe(sh6, new_buyer)

# Traffic per Country 


In [19]:
traffic_df = pd.concat([tw, sg, my, au, nz, us, hk, ph, indo, uk], ignore_index=True)
# traffic_df = pd.concat([tw, sg, my, au, nz, us, hk, ph, indo, cad, uk], ignore_index=True)
traffic_df['Date'] = pd.to_datetime(traffic_df['Date'], format = '%b %d, %Y')
traffic_df['Date'] = traffic_df['Date'].dt.strftime('%m/%d/%Y')

In [20]:

sh5 = sh.worksheet('Traffic Info')
gd.set_with_dataframe(sh5, traffic_df)

In [21]:
df_new_users = psql.read_sql('''select to_char(created_at + interval '8 hours','MM/DD/YYYY') as date,
count(id) as new_users from "user"
where to_char(created_at + interval '8 hours','MM/DD/YYYY') in %s
group by 1''' % (unique_dates_string), DbMain)


sh6 = sh.worksheet('Register Info')
gd.set_with_dataframe(sh6, df_new_users)

C:\Users\novel\AppData\Local\Temp\ipykernel_38224\2709956377.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_new_users = psql.read_sql('''select to_char(created_at + interval '8 hours','MM/DD/YYYY') as date,


In [22]:
DbMain.close()

# Campaign Specific New Buyers

In [ ]:
campaign_newbuyers = psql.read_sql('''
WITH first_time_buyers AS (
    SELECT 
        b.buyer_id,
        MIN(b.created_at + INTERVAL '8 hours') AS first_order_date,
        MIN(b.id) AS first_order_id
    FROM 
        transaction_buyer b 
    LEFT JOIN
        transaction_seller s on b.transaction_seller_id = s.id
    where b."type" = 'transaction'
    and b.status not in ('buy_failed', 'canceled','failed','payment_failed', 'sell_failed')
    and ((COALESCE(b.nature,'default') in ('Pre Order')) and procure_status not in ('check_procurement_status','null',
    'to_procure','bidding','to_complete_buy', 'contact_platform')
    or COALESCE(b.nature,'default') not in  ('Pre Order'))
    and b.payment_method!='credit'
    and buyer_id not in (12409, 20095, 1804, 19022, 87853) 
    and b.ref not in ('BW7NDNRPDG','BJ76Z59YY7','BO7L8E06EM','BPGRZ0K2KM',
    'BOM981DD67','BP716QPD5G','BPGRZ06P2M','BOM4VJEO2G','BZGE2NPP6M','BOM981D1X7','BE7DPNYYE7','BE7DPNYN17',
    'BP716QP1ZG','BE7DPNY3E7','BJ76Z5J557','BEM8YOJJDM','BOM9810L27','BJ76Z501D7','BE7DPNE007','BDG36V0NQG',
    'BKGO1NVRPG','BEMV90Y3DG','BQ7KZNV8Z7','BR7YE0YV3M','BEM8YO050M','BN7WP0Y23G','B472EX083M','B6M0W3PJ4M','BKGX8PZXV7')
    and (seller_id not in (12409, 12698) or seller_id is null)
        
    GROUP BY 
        b.buyer_id
)
SELECT 
    p.code AS campaign_code, c.name,
    COUNT(DISTINCT ftb.first_order_id) AS num_first_transactions
FROM 
    first_time_buyers ftb
JOIN 
    transaction_buyer tb ON ftb.buyer_id = tb.buyer_id
JOIN 
    promocode p ON tb.promocode_id = p.id
JOIN
    currency c ON tb.buyer_currency_id = c.id
GROUP BY 
    p.code, c.name
ORDER BY 
    p.code, c.name;
''', DbMain)

campaign_newbuyers